In [24]:
import pickle
import re

class NonOverlappingModel:

    def __init__(self):
        a_file = open("onsdurs_collapsed_cropped.pkl", "rb") #read the onsdurs file into a dictionary
        onsdurs = pickle.load(a_file)

        print('Sorting onsets and durations...')
        onsdurs_sorted = self.prioritize_events(onsdurs)

    def endtime(self, onset, duration):
            return onset + duration

    def split_runs(self, isi_times):
        '''Split runs into smaller chunks (conversations) in time'''
        chunks = []
        for i, conv_time in enumerate(isi_times):
            if i < len(isi_times) - 1:
                if isi_times[i+1] - conv_time > 50: # Check if the chunk is larger than 50 seconds, i.e., if the chunk contains a conversation.
                    chunks.append((conv_time, isi_times[i+1]))
        return chunks

    def prioritize_events(self, onsdurs):
        prioritized_order = ['^TI*', '^PROD*', '^COMP*', '^SILENCE*']
        onsdurs_sorted = {}
        for subj_run in onsdurs:
            conversations = {}
            names, ons, durs = onsdurs[subj_run].get('names'), onsdurs[subj_run].get('onsets'), onsdurs[subj_run].get('durations')
            conversation_times = self.split_runs(ons[names.index('ISI')])

            for e, conversation in enumerate(conversation_times):
                conversation_start, conversation_end = conversation[0], conversation[1]
                events = {}
                for event in prioritized_order:
                    for name in names:
                        if name == 'INSTR' or name == 'ISI':
                            continue

                        else:
                            name = ''.join(name)
                            if re.compile(event).match(name): # Check if the current name is the prioritized event
                                event_times = [(o, self.endtime(o,d)) for o, d in zip(ons[names.index([name])], durs[names.index([name])]) if conversation_start < o and self.endtime(o,d) < conversation_end]
                                events[name] = event_times

                conversations.setdefault('CONV' + str(e + 1), events)
            onsdurs_sorted.setdefault(subj_run, conversations)
        print(onsdurs_sorted)
        return onsdurs_sorted
    
m = NonOverlappingModel()
    

    

Sorting onsets and durations...
TI_h CONV1 [(17.264999999999997, 17.865), (28.263999999999996, 28.863999999999997), (37.491, 38.091), (49.903, 50.503), (65.03800000000001, 65.638), (70.601, 71.201)]
TI_r CONV1 []
PROD_h CONV1 [(19.932, 20.744999999999997), (23.016, 25.317999999999998), (28.863999999999997, 29.214999999999996), (38.091, 38.954), (52.393, 55.721000000000004)]
PROD_r CONV1 []
COMP_h CONV1 [(15.482, 17.395), (26.613999999999997, 28.375), (29.738, 34.443), (34.804, 37.268), (39.407, 39.849), (41.205, 42.272999999999996), (44.858, 48.592), (56.422000000000004, 60.395), (61.264, 62.468), (63.397, 64.459), (68.539, 70.739), (72.207, 72.9236)]
COMP_r CONV1 []
SILENCE_h CONV1 [(14.262, 15.482), (17.395, 17.865), (18.144, 19.932), (20.744999999999997, 23.016), (25.317999999999998, 26.613999999999997), (28.375, 28.863999999999997), (29.214999999999996, 29.642), (34.443, 34.804), (37.268, 38.091), (38.954, 39.407), (39.849, 41.205), (42.272999999999996, 44.858), (48.592, 50.503), (